To-do:
- " Neprošlo opravou po digitalizaci !" a absence data v 1990snr_stenprot_023schuz_s023008.htm
- Někde, např. zde: [https://www.psp.cz/eknih/1954ns/stenprot/021schuz/s021001.htm](https://www.psp.cz/eknih/1954ns/stenprot/021schuz/s021001.htm) není mluvčí označen jménem, jen jako "Předseda".
- Brajgl v 1920ns_ps_stenprot_182schuz_s182001.htm.
- Procedurální věci v 1945pns_stenprot_003schuz_s003001.htm

In [41]:
import os
import csv
import json
import random
from bs4 import BeautifulSoup
import concurrent.futures

In [42]:
kam = "data_raw/schuze"

try:
    os.makedirs(kam)
except:
    pass

In [43]:
import re

In [44]:
# re.search(r"\d{1,2}\. \w{4,12} [12]\d{3}", "zxz").group()

In [45]:
mesice = {
    'ledna':1,
    'února':2,
    'března':3,
    'dubna':4,
    'května':5,
    'června':6,
    'července':7,
    'srpna':8,
    'září':9,
    'října':10,
    'listopadu':11,
    'prosince':12
}

funkce = [
    'předseda',
    'předsedkyně',
    'předs.',
    'místopředseda',
    'místopředsedkyně',
    'podpředseda',
    'podpředsedkyně'
    'poslanec',
    'poslankyně',
    'posl.',
    'ministr',
    'ministryně',
    'zpravodaj',
    'zpravodajka',
    'zástupce',
    'zástupkyně',
    'sen.',
    'senátor',
    'senátorka',
    'společná',
    'společná',
    'tajemník',
    'tajemnice',
    'ministerský',
    'ministerská',
    'guvernér'
]

In [46]:
from collections import OrderedDict

In [47]:
def oscrapovani_stenozaznamu(soubor, zdroj=False):

    if "/" in soubor:
        soubor = soubor.replace("/","_")
    
    def ziskej_metadata(soubor):
        if "psse" not in soubor:
            komora_kratce = soubor.split("_")[0][4:]
        else:
            komora_kratce = "psse"
        obdobi = soubor.split("_")[0][0:4]
        schuze = soubor.split("_")[-1][1:4]
        cast = soubor.split("_")[-1][-7:-4]

        return komora_kratce, int(obdobi), int(schuze), int(cast.removesuffix("."))

    def najdi_datum(polivka):

        surove_datum = ""
        if polivka.find(class_="date"):
            surove_datum = polivka.find(class_="date").text.replace('\xa0',' ').lower()
        if surove_datum == "":
            try:
                surove_datum = re.search(r"\d{1,2}\. \w{4,12} [12]\d{3}", polivka.text.split("Minulý")).group()
                if zdroj == True:
                    print("Vytahuji surové datum z textu.")
            except:
                surove_datum = polivka.find("a", href="../index.htm").text.lower()

        try:
            rok = int(surove_datum.split(" ")[-1])
            mesic = surove_datum.split(" ")[-2]        
            den = int(surove_datum.split(" ")[-3].removesuffix('.'))
            return f"{rok}-{mesice[mesic]:02}-{den:02}"
        except:
            return None

    def najdi_komoru_a_schuzi(polivka):
        odkazy = polivka.find_all('a')
        komora, schuze = None, None
        try:
            komora = soup.find("h1").text.strip()
        except:
            komora = soup.find(id="breadcrumb").find_all("a")[1].text.strip()
        finally:
            pass
        for a in odkazy:
            if 'href' in a.attrs:
                if './index.htm' in a['href']:
                    schuze = a.text.strip()
        return komora, schuze

    def ciste_odstavce(seznam):
        return ' '.join(seznam).replace('\xa0',' ').replace('***','').removeprefix(':').strip()

    def serad_metadata(slovnik):
        poradi_metadat = ['komora','obdobi','schuze','cast','soubor','datum','komora_komplet','schuze_komplet','prepsano','autorizovano']
        vystupni_slovnik = OrderedDict()
        for p in poradi_metadat:
            try:
                vystupni_slovnik[p] = slovnik[p]
            except:
                vystupni_slovnik[p] = None
        return vystupni_slovnik
    
    metadata = ziskej_metadata(soubor)

    slovnik_metadata = {
        'komora': metadata[0],
        'obdobi': metadata[1],
        'schuze': metadata[2],
        'cast': metadata[3],
        'soubor': soubor
    }

    if "1996ps" in soubor:
        parsovatko = "html5lib" # kvůli 1996ps_stenprot_016schuz_s016005.htm aj., odkazy začínají mimo odstavce
    else:
        parsovatko = "lxml"
    
    with open(os.path.join("downloads/schuze/",soubor), "r", encoding="windows-1250") as schuze:
        schuze = schuze.read().replace('\n',' ')
        soup = BeautifulSoup(schuze, parsovatko)

    projevy = []
    
    if "nebyl dosud přepsán. Prosím, zkuste později." in soup.text:
        slovnik_metadata['prepsano'] = False
        return serad_metadata(slovnik_metadata), projevy
        
    if ("Neprošlo jazykovou korekturou, neautorizováno" in soup.text) or ("Neprošlo opravou po digitalizaci" in soup.text):
        slovnik_metadata['autorizovano'] = False

    
    slovnik_metadata['datum'] = najdi_datum(soup)
    slovnik_metadata['komora_komplet'], slovnik_metadata['schuze_komplet'] = najdi_komoru_a_schuzi(soup)

    if "NS RČS" in soup.find(id="breadcrumb").text:
        if "Senát" in [a.text for a in soup.find(id="breadcrumb").find_all('a')]:
            slovnik_metadata['komora_komplet'] += ", Senát"
            slovnik_metadata['komora'] = "se"
        elif "Poslanecká sněmovna" in [a.text for a in soup.find(id="breadcrumb").find_all('a')]:
            slovnik_metadata['komora_komplet'] += ", Poslanecká sněmovna"
            slovnik_metadata['komora'] = "ps"
    
    slovnik_metadata['prepsano'] = True
    slovnik_metadata['autorizovano'] = True

    orez = 130
            
    uz_byl_tucnej = False
    poradi = 1
    projev = {'mluvci':None,'poradi':poradi,'soubor':soubor,'mluvci_id':None}
    odstavce = []

    vsechny_odstavce = soup.find_all("p")

    zacatek_dokumentu = 5
    for index, p in enumerate(vsechny_odstavce[0:7]):
        if ("Minulý" in p.text) and ("Dolů" in p.text) and ("Další" in p.text) and (len(p.text.strip()) < 60):
            zacatek_dokumentu = index + 1
    
    for p in vsechny_odstavce[zacatek_dokumentu:-10]:
        if zdroj == True:
            print("____" + p.text[0:orez])

        if ("1992fs_sl" in soubor) or ('1992fs_sn' in soubor)  or ("1990fs_slsn" in soubor): # ("1990fs_sl" in soubor)

            if (':' in p.text.strip()[0:orez - 30]) and (': "' not in p.text.strip()[0:orez - 28]) and (len([l for l in p.text.strip().split(":")[0] if l.isupper()]) > 1) and (len(p.text.split(":")) > 1):

    
                
                if 'A' <= p.text.split(':')[1].strip() <= 'Ž': 
                
                    projev['text'] = ciste_odstavce(odstavce)
                    if projev['text'] != '':
                        poradi += 1
                        projevy.append(projev)
                    projev = {}
                    odstavce = []
                    uz_byl_tucnej = True
                    projev['poradi'] = poradi
                    projev['soubor'] = soubor
                    projev['mluvci'] = p.text.split(':')[0].strip()
                    projev['mluvci_id'] = None
                    odstavce.append(' '.join(p.text.split(':')[1:]).strip())

                else:

                    odstavce.append(p.text.strip())

            else:
                odstavce.append(p.text.strip())

        else:

            if zdroj:
                if p.find("b"):
                    print("TUČNÉ")

            if ':' in p.text.strip()[0:orez] and (
                    p.find('b', string=lambda text: (text.strip() in p.text.strip()[0:orez]) and (len(text.strip()) >= 3) if text else False) 
                    or 
                    p.find('u', string=lambda text: (text.strip() in p.text.strip()[0:orez]) and (len(text.strip()) >= 3) if text else False)
                ) and (
                    "výsledkem:" not in p.text # kvůli 2006se_stenprot_015schuz_s015018.htm
                ):

                pocitadlo_zvyrazneni = 0
                prvni_tucne_je_druhe_tucne = False
                zadna_zminka_o_funkci = True
                
                if len(p.find_all("b")) > 1: # kvůli 1920ns_se_stenprot_195schuz_s195001.htm
                    if zdroj == True:
                        print("počítám tučnost")
                    for tucne in p.find_all("b"):
                        if tucne.text.strip() in p.text.strip()[0:orez]:
                            pocitadlo_zvyrazneni += 1
                    if p.find_all("b")[0].text.strip() == p.find_all("b")[1].text.strip(): # kvůli 1996ps_stenprot_021schuz_s021021.htm
                        prvni_tucne_je_druhe_tucne = True
                    for f in funkce:
                        if f == p.text.strip().split(' ')[0].strip().lower(): # kvůli 1948ns_stenprot_054schuz_s054003.htm: Podpředseda Valo: Ďalším prihláseným rečníkom je pán posl. Navračič. Prosím, aby sa ujal slova.
                            if zdroj == True:
                                print(f.upper())
                            zadna_zminka_o_funkci = False
                if (pocitadlo_zvyrazneni > 1) & (prvni_tucne_je_druhe_tucne == False) & (zadna_zminka_o_funkci == True):
                    if zdroj == True:
                        print("dvakrát, bez akce")
                    odstavce.append(p.text.strip())

                else:
                    if zdroj == True:
                        print('mluvčí (: + a/b)')

                    if (("resoluční návrh" in p.text.lower()) or ("pozměňovací návrh" in p.text.lower()) or ("doplňovací návrh" in p.text.lower())) and ("192" in soubor):
                        odstavce.append(p.text.strip())
                        # kvůli 1920ns_ps_stenprot_078schuz_s078006.htm

                    else:
                        
                        projev['text'] = ciste_odstavce(odstavce)
                        if projev['text'] != '':
                            poradi += 1
                            projevy.append(projev)
                        projev = {}
                        odstavce = []
                        projev['poradi'] = poradi
                        projev['soubor'] = soubor
                        projev['mluvci'] = p.text.split(':')[0].strip()
                        try:
                            projev['mluvci_id'] = p.find("a")['href']
                        except:
                            if zdroj == True:
                                print('nenalezeno id mluvčích')
                            projev['mluvci_id'] = None
                        odstavce.append(p.text.replace(projev['mluvci'],'').strip())

            elif p.find("b") and (len(p.text.strip()) < 100) and (p.text.strip().endswith(':')):
                projev['text'] = ciste_odstavce(odstavce)
                if projev['text'] != '':
                    poradi += 1
                    projevy.append(projev)
                projev = {}
                odstavce = []
                projev['poradi'] = poradi
                projev['soubor'] = soubor
                projev['mluvci'] = p.find('b').text.replace(':','').strip()
    
            elif p.find('a'): ## častá chyba, viz testy: je odkaz, není dvojtečka. kód se tu zbytečně duplikuje, whatever
                if zdroj == True:
                    print('ODKAZ')
                if p.text.strip().startswith(p.find('a').text.strip()) and (p.find("a").get("id") != "_d") and(p.find("a").get("name") != "OLE_LINK1"):
                    if p.find("a").text.strip().startswith("Hlasování"):
                        odstavce.append(p.text.strip())

                    else:
                    
                        # p.find("a").get("name") != "OLE_LINK1" kvůli 1946uns_stenprot_087schuz_s087005.htm
                        
                        if zdroj == True:
                            print('mluvčí p.find(a)')
        
                        projev['text'] = ciste_odstavce(odstavce)
                        if projev['text'] != '':
                            poradi += 1
                            projevy.append(projev)
                        projev = {}
                        odstavce = []
                        projev['poradi'] = poradi
                        projev['soubor'] = soubor
                        projev['mluvci'] = p.find("a").text.strip()
                        try:
                            projev['mluvci_id'] = p.find("a")['href']
                        except:
                            projev['mluvci_id'] = None
                        odstavce.append(p.text.replace(projev['mluvci'],'').strip())

                else:
                    odstavce.append(p.text.strip())
                    
            else:
                odstavce.append(p.text.strip())
        
    projev['text'] = ciste_odstavce(odstavce)
    projevy.append(projev)    
    
    return serad_metadata(slovnik_metadata), projevy

In [48]:
def html2data(soubor):
    
    ciste = oscrapovani_stenozaznamu(soubor)
    soubor = soubor.replace('.htm','')
    
    with open(os.path.join(kam, f'prepis_{soubor}.ndjson'), "w+", encoding="utf-8") as export:
        for projev in ciste[1]:
            export.write(json.dumps(projev) + '\n')

    with open(os.path.join(kam, f'meta_{soubor}.csv'), "w+", newline="", encoding="utf-8") as export_file:
        fieldnames = ciste[0].keys()
        writer = csv.DictWriter(export_file, fieldnames=fieldnames, quoting=csv.QUOTE_STRINGS)
        writer.writeheader()
        writer.writerow(ciste[0])

In [49]:
# oscrapovani_stenozaznamu("2021ps_stenprot_139schuz_b18300000.htm", zdroj=True)

In [50]:
html2data("1990snr_stenprot_023schuz_s023008.htm")

In [51]:
oscrapovani_stenozaznamu("1996ps_psse_stenprot_001schuz_s001003.htm",zdroj=True)

____
____ 
____ Poslanec Vojtěch Filip: Ne. Vystupuji jako poslanec, nikoliv jako předseda klubu, a nebudu potřebovat prodloužení své lhůty.
TUČNÉ
počítám tučnost
mluvčí (: + a/b)
nenalezeno id mluvčích
____ Vážený pane předsedo Poslanecké sněmovny, vážený pane předsedo Senátu, vážený pane premiére, zbytku vlády, paní a pánové, dovolte
____ Je vždycky otázkou těch, kteří diskutují, jestli jsou schopni své myšlenky říci koncentrovaně, v krátké podobě, anebo jenom mluvi
____ Poznámka druhá se týká toho, že k volbě prezidenta přistupujeme vždycky s tím, že místo té osoby, která je ve funkci, se zabýváme
____ A nyní mi dovolte říct několik krátkých slov o našem kandidátovi. Pan ing. Stanislav Fischer, CSc., se narodil 30. listopadu 1936
____ Od roku 1969 se jako československý expert zapojil do příprav družicových experimentů programu Interkosmos ve vedoucím týmu a ved
____ Kosmickou fyzikou se zabývá i nyní v Astronomickém ústavu Akademie věd České republiky, kde pracuje již 26 let. V let

(OrderedDict([('komora', 'psse'),
              ('obdobi', 1996),
              ('schuze', 1),
              ('cast', 3),
              ('soubor', '1996ps_psse_stenprot_001schuz_s001003.htm'),
              ('datum', None),
              ('komora_komplet',
               'Parlament České republiky, Poslanecká sněmovna 1996 – 1998'),
              ('schuze_komplet', '1. schůze'),
              ('prepsano', True),
              ('autorizovano', True)]),
 [{'poradi': 1,
   'soubor': '1996ps_psse_stenprot_001schuz_s001003.htm',
   'mluvci': 'Poslanec Vojtěch Filip',
   'mluvci_id': None,
   'text': 'Ne. Vystupuji jako poslanec, nikoliv jako předseda klubu, a nebudu potřebovat prodloužení své lhůty. Vážený pane předsedo Poslanecké sněmovny, vážený pane předsedo Senátu, vážený pane premiére, zbytku vlády, paní a pánové, dovolte mi, abych předtím, než vám velmi krátce představím našeho kandidáta na funkci prezidenta České republiky, řekl dvě poznámky. Ta první se týká našeho jednání. Připadá 

In [52]:
oscrapovani_stenozaznamu("1990fs_slsn_stenprot_007schuz_s007024.htm", zdroj=True)

____
____ 
____ 
____ Omluvení poslanci: 
____ Sněmovny lidu: 
____ Andelová, Bláha, Bobovnický, Černý A., Černý J., Dientsbier st., Dlouhý, Doksanský, Honner, Jambor, Jenerál, Kačer, Kavan, Kňažko
____ Sněmovny národů zvolení v České republice: 
____ Hladík, Hradílek, Lis, Mikule, Petrová, Sokol, Stome, Šolc, Tichý 
____ Sněmovny národů zvolení ve Slovenské republice: 
____ Čalfa, Dubček, Hacaj, Hrivík, Mišura, Molnár, Moric, Muška, Pánis, Poslúch, Suchán, Šútovec, Tahy, Zelenay  
____ (Řízení schůze převzal a třetí den jednání zahájil místopředseda FS J. Stank) 
____ Předsedající místopředseda FS. J. Stank: Vážené poslankyne, vážení poslanci, otváram tretí deň 7. spoločnej schdze SĽ a SN. Praje
____ Společný zpravodaj výborů SL poslanec J. Jurečka: Dobrý den pane předsedající, dámy a pánové, na základě včerejší rozpravy v obou
____ Řada připomínek nebyla do variant zahrnuta. Myslíme si, že bychom mohli postupovat takto: Chtěli bychom se jako zpravodajové podě
____ Nyní bychom se na t

(OrderedDict([('komora', 'fs'),
              ('obdobi', 1990),
              ('schuze', 7),
              ('cast', 24),
              ('soubor', '1990fs_slsn_stenprot_007schuz_s007024.htm'),
              ('datum', '1990-10-25'),
              ('komora_komplet',
               'Federální shromáždění ČSFR, 1990 – 1992 Společné schůze SL a SN'),
              ('schuze_komplet', '7. schůze'),
              ('prepsano', True),
              ('autorizovano', True)]),
 [{'mluvci': None,
   'poradi': 1,
   'soubor': '1990fs_slsn_stenprot_007schuz_s007024.htm',
   'mluvci_id': None,
   'text': 'Omluvení poslanci: Sněmovny lidu: Andelová, Bláha, Bobovnický, Černý A., Černý J., Dientsbier st., Dlouhý, Doksanský, Honner, Jambor, Jenerál, Kačer, Kavan, Kňažko, Kocáb, Konečný J., Kováč, Langoš, Malina, Miller, Rynda, Sychra, Šepetka, Šilhán, Šimko, Tvrdá, Vidím, Zahrádka, Záležáková, Zukal Sněmovny národů zvolení v České republice: Hladík, Hradílek, Lis, Mikule, Petrová, Sokol, Stome, Šolc, Tichý 

In [53]:
oscrapovani_stenozaznamu("1990fs_slsn_stenprot_019schuz_s019110.htm", zdroj=True)

____
____ Poslanec SN J. Sokol: Vážené Federální shromáždění, vážení hosté, několik krátkých poznámek. Kolega Kříž bohužel opustil jednací 
____ Společnost může být demokratická jen do té míry, do jaké míry dokáže demokraticky hospodařit se svými penězi. Je to věc tak banál
____ A to mě vede k dalším poznámkám. Často se hovořívá o populismu, jindy o demagogii. Tyhle věci mají něco společného. Spočívají vžd
____ Dovolte, abych to i zde demonstroval na několika příkladech. V každé rozpočtové debatě se vždycky dojde k úsporám na státní správ
____ Mluvilo se tu několikrát o tom - a to je věc, které bych se rád trochu věnoval - že určité škrty v rozpočtu se dějí na úkor obyva
____ Představa, že nějaké výdaje státního rozpočtu se dějí na úkor obyvatelstva, mi připadá obludná. A pokud se týče návrhu na změny v
____ A to je přesně to, co stát nikdy dělat nemá. Přece stát je od toho, aby vyrovnával tam, kde je to potřeba, ale v žádném případě s
____ A konečně poslední poznámka. Hovořilo se tady

(OrderedDict([('komora', 'fs'),
              ('obdobi', 1990),
              ('schuze', 19),
              ('cast', 110),
              ('soubor', '1990fs_slsn_stenprot_019schuz_s019110.htm'),
              ('datum', '1991-12-20'),
              ('komora_komplet',
               'Federální shromáždění ČSFR, 1990 – 1992 Společné schůze SL a SN'),
              ('schuze_komplet', '19. schůze'),
              ('prepsano', True),
              ('autorizovano', True)]),
 [{'poradi': 1,
   'soubor': '1990fs_slsn_stenprot_019schuz_s019110.htm',
   'mluvci': 'Poslanec SN J. Sokol',
   'mluvci_id': None,
   'text': 'Vážené Federální shromáždění, vážení hosté, několik krátkých poznámek. Kolega Kříž bohužel opustil jednací místnost. Chtěl jsem se zastat jednoho z našich předků, kronikáře Kosmy. Označit Kosmu za servilního považuji za nehoráznost. (Potlesk a klepání do lavic.) Společnost může být demokratická jen do té míry, do jaké míry dokáže demokraticky hospodařit se svými penězi. Je to věc t

In [54]:
oscrapovani_stenozaznamu("1990fs_slsn_stenprot_004schuz_s004013.htm", zdroj=True)

____
____ Předsedající předseda SN M. Šútovec: Nerád prerušujem váš mravný apel, ale máme ešte niekoľko prihlášok do diskusie a prosil by s
____ Poslanec SL R. Sacher: Vážený pane předsedo, vážený pane předsedající, vážené dámy a pánové, budu se držet pokynů a budu stručný.
____ Dovolte několik slov k programovému prohlášení. Myslím, že by bylo třeba v programovém prohlášení zcela jasně a bez obalu říci na
____ V podstatě dosud žádná budova zvýšeného zájmu nebo ohrožení není vybavena detekčním rámem. V podstatě nevíme, co jsou to širokosp
____ Co říci dále. Platy policie. Nedávno zde byli zástupci protiteroristického amerického komanda. Srovnávali úroveň našeho protitero
____ Domnívám se, že by bylo vhodné, aby obdobným způsobem, jako úřad pro ochranu ústavy a demokracie, byla vyčleněna z ministerstva v
____ Dále mi dovolte, abych se zmínil, jak zde bylo řečeno několika slovy na adresu pana M. Čalfy. Účastnil jsem se jednání o vytvořen
____ Předsedající předseda SN M. Šútovec: Ďakujem 

(OrderedDict([('komora', 'fs'),
              ('obdobi', 1990),
              ('schuze', 4),
              ('cast', 13),
              ('soubor', '1990fs_slsn_stenprot_004schuz_s004013.htm'),
              ('datum', '1990-07-10'),
              ('komora_komplet',
               'Federální shromáždění ČSFR, 1990 – 1992 Společné schůze SL a SN'),
              ('schuze_komplet', '4. schůze'),
              ('prepsano', True),
              ('autorizovano', True)]),
 [{'poradi': 1,
   'soubor': '1990fs_slsn_stenprot_004schuz_s004013.htm',
   'mluvci': 'Předsedající předseda SN M. Šútovec',
   'mluvci_id': None,
   'text': 'Nerád prerušujem váš mravný apel, ale máme ešte niekoľko prihlášok do diskusie a prosil by som aj nasledujúcich diskutérov, aby sa trošku tesnejšie držali predmetné témy. Ďakujem. Prosím pána poslanca Sachera.'},
  {'poradi': 2,
   'soubor': '1990fs_slsn_stenprot_004schuz_s004013.htm',
   'mluvci': 'Poslanec SL R. Sacher',
   'mluvci_id': None,
   'text': 'Vážený pane p

In [55]:
oscrapovani_stenozaznamu("2006ps_stenprot_048schuz_s048369.htm", zdroj=True)

____
____ 
____(9.30 hodin) (pokračuje Talmanová)
____ Můžeme přistoupit k prvnímu pevně zařazenému bodu, kterým je bod
____ 
____222. Vládní návrh zákona, kterým se mění zákon č. 235/2004 Sb.,  o dani z přidané hodnoty, ve znění pozdějších předpisů, a zákon  
ODKAZ
____ 
____ U stolku zpravodajů již zaujali svá místa za navrhovatele ministr financí pan Miroslav Kalousek a zpravodaj rozpočtového výboru, 
____ Souhlasili jsme s návrhem na opakování druhého čtení tohoto návrhu zákona, a to usnesením Poslanecké sněmovny č. 1085 ze dne 18. 
____ V úvodu vystoupí ministr financí pan Miroslav Kalousek. Vážený pane ministře, máte slovo. 
____ 
____ Ministr financí ČR Miroslav Kalousek  Děkuji za slovo, vážená paní předsedající. Dámy a pánové, již počtvrté předstupuji před Pos
TUČNÉ
ODKAZ
mluvčí p.find(a)
____ Ke všem ostatním pozměňovacím návrhům, popřípadě i k tomuto, jsem připraven vyjádřit se před třetím čtením. Děkuji za pozornost. 
____ 
____ Místopředsedkyně PSP Lucie Talmanová: I já dě

(OrderedDict([('komora', 'ps'),
              ('obdobi', 2006),
              ('schuze', 48),
              ('cast', 369),
              ('soubor', '2006ps_stenprot_048schuz_s048369.htm'),
              ('datum', '2009-02-20'),
              ('komora_komplet',
               'Parlament České republiky, Poslanecká sněmovna 2006 – 2010'),
              ('schuze_komplet', '48. schůze'),
              ('prepsano', True),
              ('autorizovano', True)]),
 [{'mluvci': None,
   'poradi': 1,
   'soubor': '2006ps_stenprot_048schuz_s048369.htm',
   'mluvci_id': None,
   'text': '(9.30 hodin) (pokračuje Talmanová) Můžeme přistoupit k prvnímu pevně zařazenému bodu, kterým je bod  222. Vládní návrh zákona, kterým se mění zákon č. 235/2004 Sb.,  o dani z přidané hodnoty, ve znění pozdějších předpisů, a zákon  č. 353/2003 Sb., o spotřebních daních, ve znění pozdějších předpisů  /sněmovní tisk 605/ - druhé čtení  U stolku zpravodajů již zaujali svá místa za navrhovatele ministr financí pan Miro

In [56]:
oscrapovani_stenozaznamu("2021ps_stenprot_140schuz_s140050.htm", zdroj=True)

____
____ 
____Předsedkyně PSP Markéta Pekarová Adamová: Nyní tedy avizovaná paní poslankyně Peštová, která je další přihlášenou, a následuje pan
TUČNÉ
mluvčí (: + a/b)
____ 
____Poslankyně Berenika Peštová: Děkuji za slovo, paní předsedající. Každá akce vyvolá reakci. Takže jelikož jsem zjistila, že k tomut
TUČNÉ
mluvčí (: + a/b)
____Ta druhá část mého pozměňovacího návrhu se týká střetu zájmů. V současné době už zákon o obalech hovoří o střetu zájmů a jasně řík
____I ráda bych zde připomněla, dneska jsem byla dotázána, nebo - dostala jsem se do takového menšího konfliktu, kdy mi bylo vysvětlen
____ 
____Předsedkyně PSP Markéta Pekarová Adamová: Nyní tedy vystoupí pan poslanec Smetana jako poslední přihlášený v obecné rozpravě. Pros
TUČNÉ
mluvčí (: + a/b)
____ 
____Poslanec Karel Smetana: Krásný den. Já teda přicházím s akcí, která přichází až po reakci, a chtěl bych odůvodnit pozměňovací návr
TUČNÉ
mluvčí (: + a/b)
____ 
____Předsedkyně PSP Markéta Pekarová Adamová: Děkuji. Já vás, v

(OrderedDict([('komora', 'ps'),
              ('obdobi', 2021),
              ('schuze', 140),
              ('cast', 50),
              ('soubor', '2021ps_stenprot_140schuz_s140050.htm'),
              ('datum', '2025-05-29'),
              ('komora_komplet', 'PČR, PS 2021-...'),
              ('schuze_komplet', '140. schůze'),
              ('prepsano', True),
              ('autorizovano', True)]),
 [{'poradi': 1,
   'soubor': '2021ps_stenprot_140schuz_s140050.htm',
   'mluvci': 'Předsedkyně PSP Markéta Pekarová Adamová',
   'mluvci_id': '/sqw/detail.sqw?id=6147',
   'text': 'Nyní tedy avizovaná paní poslankyně Peštová, která je další přihlášenou, a následuje pan poslanec Smetana. Prosím, máte slovo.'},
  {'poradi': 2,
   'soubor': '2021ps_stenprot_140schuz_s140050.htm',
   'mluvci': 'Poslankyně Berenika Peštová',
   'mluvci_id': '/sqw/detail.sqw?id=6780',
   'text': 'Děkuji za slovo, paní předsedající. Každá akce vyvolá reakci. Takže jelikož jsem zjistila, že k tomuto sněmovnímu ti

In [57]:
oscrapovani_stenozaznamu("2021ps_stenprot_140schuz_s140050.htm", zdroj=True)

____
____ 
____Předsedkyně PSP Markéta Pekarová Adamová: Nyní tedy avizovaná paní poslankyně Peštová, která je další přihlášenou, a následuje pan
TUČNÉ
mluvčí (: + a/b)
____ 
____Poslankyně Berenika Peštová: Děkuji za slovo, paní předsedající. Každá akce vyvolá reakci. Takže jelikož jsem zjistila, že k tomut
TUČNÉ
mluvčí (: + a/b)
____Ta druhá část mého pozměňovacího návrhu se týká střetu zájmů. V současné době už zákon o obalech hovoří o střetu zájmů a jasně řík
____I ráda bych zde připomněla, dneska jsem byla dotázána, nebo - dostala jsem se do takového menšího konfliktu, kdy mi bylo vysvětlen
____ 
____Předsedkyně PSP Markéta Pekarová Adamová: Nyní tedy vystoupí pan poslanec Smetana jako poslední přihlášený v obecné rozpravě. Pros
TUČNÉ
mluvčí (: + a/b)
____ 
____Poslanec Karel Smetana: Krásný den. Já teda přicházím s akcí, která přichází až po reakci, a chtěl bych odůvodnit pozměňovací návr
TUČNÉ
mluvčí (: + a/b)
____ 
____Předsedkyně PSP Markéta Pekarová Adamová: Děkuji. Já vás, v

(OrderedDict([('komora', 'ps'),
              ('obdobi', 2021),
              ('schuze', 140),
              ('cast', 50),
              ('soubor', '2021ps_stenprot_140schuz_s140050.htm'),
              ('datum', '2025-05-29'),
              ('komora_komplet', 'PČR, PS 2021-...'),
              ('schuze_komplet', '140. schůze'),
              ('prepsano', True),
              ('autorizovano', True)]),
 [{'poradi': 1,
   'soubor': '2021ps_stenprot_140schuz_s140050.htm',
   'mluvci': 'Předsedkyně PSP Markéta Pekarová Adamová',
   'mluvci_id': '/sqw/detail.sqw?id=6147',
   'text': 'Nyní tedy avizovaná paní poslankyně Peštová, která je další přihlášenou, a následuje pan poslanec Smetana. Prosím, máte slovo.'},
  {'poradi': 2,
   'soubor': '2021ps_stenprot_140schuz_s140050.htm',
   'mluvci': 'Poslankyně Berenika Peštová',
   'mluvci_id': '/sqw/detail.sqw?id=6780',
   'text': 'Děkuji za slovo, paní předsedající. Každá akce vyvolá reakci. Takže jelikož jsem zjistila, že k tomuto sněmovnímu ti

In [58]:
oscrapovani_stenozaznamu("1981fs_slsn_stenprot_013schuz_s013001.htm")

(OrderedDict([('komora', 'fs'),
              ('obdobi', 1981),
              ('schuze', 13),
              ('cast', 1),
              ('soubor', '1981fs_slsn_stenprot_013schuz_s013001.htm'),
              ('datum', '1984-06-20'),
              ('komora_komplet',
               'Federální shromáždění ČSSR, 1981 – 1986 Společné schůze SL a SN'),
              ('schuze_komplet', '13. schůze'),
              ('prepsano', True),
              ('autorizovano', True)]),
 [{'mluvci': None,
   'poradi': 1,
   'soubor': '1981fs_slsn_stenprot_013schuz_s013001.htm',
   'mluvci_id': None,
   'text': 'Omluveni poslanci'},
  {'poradi': 2,
   'soubor': '1981fs_slsn_stenprot_013schuz_s013001.htm',
   'mluvci': 'Sněmovny lidu',
   'text': 'Bartončík, Bobková, Boroš, Brabenec, Humajová, Chňoupek, Kapek, Králíčková, Löbl, Mičinová, Paruza, Pitra, Seidlová, Skřivan, Zavadil'},
  {'poradi': 3,
   'soubor': '1981fs_slsn_stenprot_013schuz_s013001.htm',
   'mluvci': 'Sněmovny národů zvolení v České socialisti

In [59]:
oscrapovani_stenozaznamu("2017ps_stenprot_029schuz_s029109.htm")

(OrderedDict([('komora', 'ps'),
              ('obdobi', 2017),
              ('schuze', 29),
              ('cast', 109),
              ('soubor', '2017ps_stenprot_029schuz_s029109.htm'),
              ('datum', None),
              ('komora_komplet', None),
              ('schuze_komplet', None),
              ('prepsano', False),
              ('autorizovano', None)]),
 [])

In [60]:
oscrapovani_stenozaznamu("2017ps_stenprot_029schuz_s029109.htm", zdroj=True)

(OrderedDict([('komora', 'ps'),
              ('obdobi', 2017),
              ('schuze', 29),
              ('cast', 109),
              ('soubor', '2017ps_stenprot_029schuz_s029109.htm'),
              ('datum', None),
              ('komora_komplet', None),
              ('schuze_komplet', None),
              ('prepsano', False),
              ('autorizovano', None)]),
 [])

In [61]:
def process_file(soubor):
    try:
        html2data(soubor)
    except Exception as e:
        print(f"{soubor}: {e}")

In [ ]:
directory = "downloads/schuze"
files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
# files = [f for f in files if "199" in f]
# files = [f for f in files if ('1992fs_sl' in f) or ('1990fs_sl' in f)] 
total_files = len(files)

batch_size = 8

# Process files in batches to reduce memory usage and improve progress tracking
for i in range(0, total_files, batch_size):
    batch = files[i:i + batch_size]
    
    # Use ProcessPoolExecutor for CPU-bound tasks or ThreadPoolExecutor for I/O-bound tasks
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Map is more memory efficient than list comprehension for large batches
        futures = {executor.submit(process_file, f): f for f in batch}
        
        for j, future in enumerate(concurrent.futures.as_completed(futures), 1):
            try:
                future.result()  # Get the result to raise any exceptions
            except Exception as e:
                print(f"Error processing file {futures[future]}: {e}")
            
            # # Update progress every batch_size files
            # if (i + j) % batch_size == 0:
            #    print(f"{i + j}/{total_files}")